# 2-D CNN Example (shape classification)

In [19]:
'''
A Convolutional Network implementation example using TensorFlow library.
This example tries to classify images of square and circles.

Author: Rodrigo Ceballos

Adapted from: https://github.com/aymericdamien/TensorFlow-Examples/
'''

from __future__ import print_function

import tensorflow as tf
import pickle
from dataset_def import Dataset


In [51]:
# Parameters
learning_rate = 0.01
beta = 0.01 # multiplies with regularization term in loss function
training_iters = 20000
batch_size = 64
display_step = 10

# Where to save logs
logs_path = '/tmp/tensorflow_logs/shapes2D'

# Network Parameters
n_input = 28**2 # Data generated in data_gen notebook is 28 by 28 by default
n_classes = 2 # Squares and Circles
dropout = 0.75 # Dropout, probability to keep units

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)


In [32]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out



In [33]:
def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

In [42]:
layer_sizes = [16,32,64]
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, layer_sizes[0]])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, layer_sizes[0], layer_sizes[1]])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*layer_sizes[1], layer_sizes[2]])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([layer_sizes[2], n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([layer_sizes[0]])),
    'bc2': tf.Variable(tf.random_normal([layer_sizes[1]])),
    'bd1': tf.Variable(tf.random_normal([layer_sizes[2]])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}



In [35]:
# Load data
tf.logging.info('loading data...')
with open('data/shapes/2D_train_dataset.pickle', 'rb') as data:
    train_dataset = pickle.load(data)
    
with open('data/shapes/2D_test_dataset.pickle', 'rb') as data:
    test_dataset = pickle.load(data)
tf.logging.info('DONE')

INFO:tensorflow:loading data...
INFO:tensorflow:DONE


In [54]:
# Construct model
pred = conv_net(x, weights, biases, keep_prob)


# Regularize Weights
for name, weight in weights.iteritems():
    if 'regularizer' in locals(): regularizer += tf.nn.l2_loss(weight)
    else: regularizer = tf.nn.l2_loss(weight)
    
# Define loss and optimizer
with tf.name_scope('Loss'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y) + beta*regularizer)
    variable_summaries(cost)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
with tf.name_scope('Accuracy'):
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    variable_summaries(accuracy)


# Create a summary to monitor cost tensor
cost_summ = tf.summary.scalar("loss", cost)
# Create a summary to monitor accuracy tensor
acc_summ = tf.summary.scalar("accuracy", accuracy)
# Merge all summaries into a single op
merged = tf.summary.merge([cost_summ, acc_summ]);

# Initializing the variables
init = tf.global_variables_initializer()


In [55]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        
        batch_x, batch_y = train_dataset.next_batch(batch_size)
        # Run optimization op (backprop)
        _, summary = sess.run([optimizer, merged], feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        summary_writer.add_summary(summary, step)
        
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            test_batch_x, test_batch_y = test_dataset.next_batch(256)
            accTest = sess.run(accuracy, feed_dict={x: test_batch_x,
                                      y: test_batch_y,
                                      keep_prob: 1.})
            print("Iter " + str(step*batch_size) + \
                  ", Minibatch Loss= " + "{:.6f}".format(loss) + \
                  ", Training Accuracy= " + "{:.5f}".format(acc) + \
                  ", Testing Accuracy= " + "{:.5f}".format(accTest) )
            
            
            
            
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 256 mnist test images
    test_batch_x, test_batch_y = test_dataset.next_batch(256)
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_batch_x,
                                      y: test_batch_y,
                                      keep_prob: 1.}))
    print("Run the command line:\n" \
          "--> tensorboard --logdir=/tmp/tensorflow_logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")

Iter 1280, Minibatch Loss= 3075.438721, Training Accuracy= 0.59375, Testing Accuracy= 0.58000
Iter 2560, Minibatch Loss= 2396.412598, Training Accuracy= 0.75000, Testing Accuracy= 0.69000
Iter 3840, Minibatch Loss= 2159.945312, Training Accuracy= 0.84375, Testing Accuracy= 0.79000
Iter 5120, Minibatch Loss= 2083.622559, Training Accuracy= 0.85938, Testing Accuracy= 0.80000
Iter 6400, Minibatch Loss= 1985.552856, Training Accuracy= 0.88281, Testing Accuracy= 0.86000
Iter 7680, Minibatch Loss= 1932.157227, Training Accuracy= 0.92969, Testing Accuracy= 0.85000
Iter 8960, Minibatch Loss= 1868.141113, Training Accuracy= 0.95312, Testing Accuracy= 0.86000
Iter 10240, Minibatch Loss= 1817.896606, Training Accuracy= 0.94531, Testing Accuracy= 0.87000
Iter 11520, Minibatch Loss= 1765.983154, Training Accuracy= 1.00000, Testing Accuracy= 0.87000
Iter 12800, Minibatch Loss= 1724.656250, Training Accuracy= 0.98438, Testing Accuracy= 0.91000
Iter 14080, Minibatch Loss= 1685.036377, Training Accurac

### DANGER: Deletes logged data

In [58]:
import os, shutil
def deleteFolderContents(folder_path, delete_sub_dir = False):
    for the_file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif delete_sub_dir and os.path.isdir(file_path): shutil.rmtree(file_path)
        except Exception as e:
            print(e)
    

In [1]:
# CAREFUL: DELETE ALL LOGS
folder = '/tmp/tensorflow_logs/'
deleteFolderContents(folder)

In [38]:
# CAREFUL, DELETE THIS LOG (need to run)
deleteFolderContents(logs_path)